# Getting data from spotify for RecNet

In [ ]:
import pandas as pd
import requests
from requests.structures import CaseInsensitiveDict
import json

pd.options.mode.chained_assignment = None  # default='warn'

### Update your username below. This is just to identify that its your data, so it can be anything. In the actual implementation this can be replaced with RecNet username.

In [ ]:
username = 'neel'

### Step 1: Download a csv file from https://watsonbox.github.io/exportify/, and assign the path variable with its appropriate location.

In [ ]:
path = 'liked.csv'
df = pd.read_csv(path)
print("Downloaded data:")
df

In [ ]:
print("Relevant data: ")
data = df[['Track URI', 'Track Name', 'Artist URI(s)', 'Artist Name(s)', 'Popularity']]
for i, row in data.iterrows():
    data.at[i, 'Artist'] = row['Artist Name(s)'].split(',')[0]
    data.at[i, 'Other Artists'] = ','.join(row['Artist Name(s)'].split(',')[1:])
data

### Assign the genereted on Spotify console below (as a string)
Generate code here: https://developer.spotify.com/console/get-audio-features-several-tracks/

Note: Make sure you copy the entire token (its kinda huge and may have a lot of hyphens, so avoid selecting using a double click, instead use cmd+A (or its equivalent))

In [1]:

token = ''


In [ ]:
track_ids = 'ids='
for row in df['Track URI']:
    track_ids += row.split(':')[2] + '%2C'
track_ids

url = "https://api.spotify.com/v1/audio-features"
url += '?' + track_ids
headers = CaseInsensitiveDict()
headers["Accept"] = "application/json"
headers["Content-Type"] = "application/json"
headers["Authorization"] = "Bearer " + token
resp = requests.get(url, headers=headers)
audio_features_json = resp.json()
audio_features_df = pd.json_normalize(audio_features_json['audio_features'])
audio_features = audio_features_df[['uri', 'danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence']]
audio_features = audio_features.rename(columns = {'uri': 'Track URI'})
audio_features

### Extracting album ids and getting genre details

In [ ]:
album_ids = []

for row in df['Album URI']:
    album_ids.append(row.split(':')[2])
    
album_ids = list(dict.fromkeys(album_ids))
batch_size = 15
batches = []
i = 0
while i < len(album_ids):
    batch = album_ids[i:i+batch_size]
    i += len(batch)
    batches.append(batch)
    
del(album_genres)
for batch in batches:
    album_ids_url = 'ids='
    for album_id in batch:
        album_ids_url += album_id + '%2C'
    
    album_ids_url = album_ids_url[0:len(album_ids_url) - 3]
    url = "https://api.spotify.com/v1/albums"
    url += '?' + album_ids_url
    
    headers = CaseInsensitiveDict()
    headers["Accept"] = "application/json"
    headers["Content-Type"] = "application/json"
    headers["Authorization"] = "Bearer " + token
    resp = requests.get(url, headers=headers)
    album_details_json = resp.json()

    album_details_df = pd.json_normalize(album_details_json['albums'])
    try:
        album_genres
    except NameError:
        album_genres = album_details_df[['uri', 'genres']]
        album_genres = album_genres.rename(columns = {'uri': 'Album URI'})
    else:
        album_genres2 = album_details_df[['uri', 'genres']]
        album_genres2 = album_genres2.rename(columns = {'uri': 'Album URI'})
        album_genres = pd.concat([album_genres, album_genres2],ignore_index=True)
album_genres


## Merging all the retrieved data and saving it to username_spotifydata.csv.

In [ ]:
data = df[['Track URI', 'Album URI', 'Track Name', 'Artist URI(s)', 'Artist Name(s)', 'Popularity']]
data = pd.merge(data, audio_features, on = ['Track URI'])
data = pd.merge(data, album_genres, on=['Album URI'])
data['Username'] = username
data

In [ ]:
data.to_csv(username + 'spotifydata.csv')